# Shor 9-Qubit Quantum Error Correction Code

This notebook demonstrates the Shor 9-Qubit Quantum Error Correction Code from Chapter 13, Quantum Error Correction: A Primer.

## 0. Setup

The cells below install the packages and import the libraries needed by this demo.

In [ ]:
#@title Install Qiskit and other dependencies
%pip install qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [2]:
# Imports

import math
from IPython.display import display, HTML, clear_output
from ipywidgets import interact
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit import  ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import ZGate
from qiskit.quantum_info import random_unitary
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

## 1. Circuit Definition

The following cell defines various implementations of the Shor 9-Qubit Quantum Error Correction Code circuit.

While this circuit maintains the same functionality as Figure 13.15 in the QCI textbook, its layout differs for programming convenience. In this implementation, the data qubits are arranged sequentially as $q_1, q_2, \ldots, q_9$ and the ancilla qubits as $a_1, a_2, \ldots, a_8$. In contrast, Figure 13.15 uses the arrangement $q_1, q_2, q_3, a_1, a_2$; $q_4, q_5, q_6, a_3, a_4$; $q_7, q_8, q_9, a_5, a_6$, followed by $a_7, a_8$ for visual clarity.

Refer to **2. Circuit Diagram** for configurable options (parameters).


In [3]:
def create_circuit(rx=0.0, unitaryop=None, xbit=-1, zbit=-1, randombit=-1, deferred=True):
    q = [q1, q2, q3, q4, q5, q6, q7, q8, q9] = QuantumRegister(9, name='q')
    a = [a1, a2, a3, a4, a5, a6, a7, a8] = QuantumRegister(8, name='a')
    o = ClassicalRegister(9, name='output')
    s = [s1, s2, s3, s4, s5, s6, s7, s8] = ClassicalRegister(8, name='syndrome')
    circuit = QuantumCircuit(q, a, o, s)

    if rx != 0.0:
        circuit.rx(rx, q1)
    # encode phase flip
    circuit.cx(q1, q4)
    circuit.cx(q1, q7)
    circuit.h([q1, q4, q7])
    # encode bit flips
    circuit.cx(q1, q2)
    circuit.cx(q1, q3)
    circuit.cx(q4, q5)
    circuit.cx(q4, q6)
    circuit.cx(q7, q8)
    circuit.cx(q7, q9)
    circuit.barrier()
    if unitaryop == 'X':
        circuit.z(q1)
        circuit.z(q4)
        circuit.z(q7)
    if unitaryop == 'Z':
        circuit.x(q1)
        circuit.x(q2)
        circuit.x(q3)
    circuit.barrier()
    if xbit != -1:
        circuit.x(q[xbit])
    if zbit != -1:
        circuit.z(q[zbit])
    if randombit != -1:
        random_rotation = random_unitary(2)
        circuit.append(random_rotation, [q[randombit]])
    circuit.barrier()
    # bit flips correction
    circuit.cx(q1, a1)
    circuit.cx(q2, a1)
    circuit.cx(q1, a2)
    circuit.cx(q3, a2)
    circuit.barrier()
    circuit.cx(q4, a3)
    circuit.cx(q5, a3)
    circuit.cx(q4, a4)
    circuit.cx(q6, a4)
    circuit.barrier()
    circuit.cx(q7, a5)
    circuit.cx(q8, a5)
    circuit.cx(q7, a6)
    circuit.cx(q9, a6)
    circuit.barrier()
    if not deferred:
        circuit.measure([a1, a2, a3, a4, a5, a6], [s1, s2, s3, s4, s5, s6])
        with circuit.if_test((s1, 1)):
            with circuit.if_test((s2, 1)):
                circuit.x(q1)
        with circuit.if_test((s1, 1)):
            with circuit.if_test((s2, 0)):
                circuit.x(q2)
        with circuit.if_test((s1, 0)):
            with circuit.if_test((s2, 1)):
                circuit.x(q3)
        circuit.barrier()
        with circuit.if_test((s3, 1)):
            with circuit.if_test((s4, 1)):
                circuit.x(q4)
        with circuit.if_test((s3, 1)):
            with circuit.if_test((s4, 0)):
                circuit.x(q5)
        with circuit.if_test((s3, 0)):
            with circuit.if_test((s4, 1)):
                circuit.x(q6)
        circuit.barrier()
        with circuit.if_test((s5, 1)):
            with circuit.if_test((s6, 1)):
                circuit.x(q7)
        with circuit.if_test((s5, 1)):
            with circuit.if_test((s6, 0)):
                circuit.x(q8)
        with circuit.if_test((s5, 0)):
            with circuit.if_test((s6, 1)):
                circuit.x(q9)
        circuit.barrier()
    else:
        circuit.mcx([a2, a1], q1, ctrl_state=0b11)
        circuit.mcx([a2, a1], q2, ctrl_state=0b10)
        circuit.mcx([a2, a1], q3, ctrl_state=0b01)
        circuit.barrier()
        circuit.mcx([a4, a3], q4, ctrl_state=0b11)
        circuit.mcx([a4, a3], q5, ctrl_state=0b10)
        circuit.mcx([a4, a3], q6, ctrl_state=0b01)
        circuit.barrier()
        circuit.mcx([a6, a5], q7, ctrl_state=0b11)
        circuit.mcx([a6, a5], q8, ctrl_state=0b10)
        circuit.mcx([a6, a5], q9, ctrl_state=0b01)
        circuit.barrier()
    # decode bitflips: note this is in a different order from the first edition of the book.
    circuit.cx(q7, q9)
    circuit.cx(q7, q8)
    circuit.cx(q4, q6)
    circuit.cx(q4, q5)
    circuit.cx(q1, q3)
    circuit.cx(q1, q2)
    circuit.barrier()
    # phase flip correction
    circuit.h([q1, q4, q7])
    circuit.cx(q1, a7)
    circuit.cx(q4, a7)
    circuit.cx(q1, a8)
    circuit.cx(q7, a8)
    circuit.barrier()
    circuit.h([q1, q4, q7])
    if not deferred:
        circuit.measure([a7, a8], [s7, s8])
        with circuit.if_test((s7, 1)):
            with circuit.if_test((s8, 1)):
                circuit.z(q1)
        with circuit.if_test((s7, 1)):
            with circuit.if_test((s8, 0)):
                circuit.z(q4)
        with circuit.if_test((s7, 0)):
            with circuit.if_test((s8, 1)):
                circuit.z(q7)
        circuit.barrier()
    else:
        circuit.append(ZGate().control(num_ctrl_qubits=2, ctrl_state=0b11), [a8, a7, q1])
        circuit.append(ZGate().control(num_ctrl_qubits=2, ctrl_state=0b10), [a8, a7, q4])
        circuit.append(ZGate().control(num_ctrl_qubits=2, ctrl_state=0b01), [a8, a7, q7])
        circuit.barrier()
    # decode phase flip
    circuit.h([q1, q4, q7])
    circuit.cx(q1, q7)
    circuit.cx(q1, q4)
    circuit.barrier()
    circuit.measure(q, o)
    if deferred:
        circuit.measure(a, s)
    return circuit

## 2. Circuit Diagram

The cell below displays variants of the circuit diagram of the Shor 9-Qubit Quantum Error Correction Code defined in **1. Circuit Definition**.

You can experiment with these parameters:

* `rx`: The RX angle for the initialization state.
* `unitaryop`: The logical unitary operation to apply to the encoded state: None, $X_L$, or $Z_L$.
* `xbit`: Flip a qubit with an $X$ gate: -1 = no flip, 0-8 = flip qubit 0-8.
* `zbit`: Flip a qubit with a $Z$ gate: -1 = no flip, 0-8 = flip qubit 0-8.
* `randombit`: Apply a random rotation to a qubit: -1 = no rotation, 0-8 = rotate qubit 0-8.
* `deferred`: Build the circuit with deferred measurement, or not.

Remember, the Shor 9-bit code can correct an arbitrary error on only a single qubit!

In [ ]:
def draw_circuit(rx=0.0, unitaryop=None, xbit=-1, zbit=-1, randombit=-1, deferred=True):
    circuit = create_circuit(rx, unitaryop, xbit, zbit, randombit, deferred)
    display(circuit.draw(output="mpl", fold=-1, plot_barriers=False, cregbundle=True))

interact(draw_circuit, rx=(0.0, math.pi, math.pi/8), unitaryop=[None, 'X', 'Z'], xbit=(-1, 8), zbit=(-1, 8), randombit=(-1, 8), deferred=True)

## 3. Circuit Results

The cell below displays the results of running the Shor 9-Qubit Quantum Error Correction Code circuit.

You can experiment with the same parameters as in **2. Circuit Diagram**.

In [ ]:


def run_circuit(rx=0.0, unitaryop=None, xbit=-1, zbit=-1, randombit=-1, deferred=True):
    clear_output(wait=True)
    display(HTML("<span style='color: #ff5733;'>Running, please wait...</span>")) 
    circuit = create_circuit(rx, unitaryop, xbit, zbit, randombit, deferred)
    backend = AerSimulator()
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    circuit = pm.run(circuit)
    sampler = Sampler(backend)
    job = sampler.run([circuit], shots=1000)
    result = job.result()

    clear_output(wait=True)
    syndrome_counts = result[0].data.syndrome.get_counts()
    if randombit == -1:
        assert len(syndrome_counts) == 1
    print(f"{'Syndrome':<10} {'counts':<6}") 
    for bits, val in sorted(syndrome_counts.items()):
        print(f"{bits:<10} {val:<6}")
    output_counts = result[0].data.output.get_counts()
    print(f"\n{'Output':<10} {'counts':<6}")
    for bits, val in sorted(output_counts.items()):
        assert bits[0:8] == "00000000"
        bit = bits[8]
        print(f"{bit:<10} {val:<6}")

interact(run_circuit, rx=(0.0, math.pi, math.pi/8), unitaryop=[None, 'X', 'Z'], xbit=(-1, 8), zbit=(-1, 8), randombit=(-1, 8), deferred=True)

## 4. Alternative Circuit Definition

The following cell defines various implementations of the Shor 9-Qubit Quantum Error Correction Code circuit, directly mirroring Figure 13.15 in the *Quantum Computing and Information* textbook.

In this circuit, the data qubits and ancilla qubits are interleaved as $q_1, q_2, q_3, a_1, a_2$; $q_4, q_5, q_6, a_3, a_4$; $q_7, q_8, q_9, a_5, a_6$, followed by $a_7, a_8$ for visual clarity. In **1. Circuit Definition**, they are arranged as $q_1, q_2, \ldots, q_9$, $a_1, a_2, \ldots, a_8$ for programming convenience.

Refer to **2. Circuit Diagram** for configurable options (parameters).


In [ ]:
# Code TODO

## 5. Simplified Circuit

The Shor code can also be applied to quantum communication, where quantum states are transmitted in the presence of noise. In this case, repeating the middle steps is often unnecessary, allowing for a simplified circuit, as shown in Fig. 13.16 in the QCI Book. In this simplified version, the error correction process can be postponed until after decoding, eliminating the need for ancillary qubits. 


In [ ]:
# Code TODO

## 6. Exercise

Your tasks: Display the circuit diagrams and verify their operations for the circuits defined in **4. Alternative Circuit Definition** and **5. Simplified Circuit**.

In [ ]:
# Code to be provided by readers.